# Multilayer Convolutional Network (MNIST 手寫數字辨識)
2017/06/26  
徐仕杰 

In [1]:
import numpy as np
from PIL import Image
import lzma
import pickle
from IPython.display import display
import tensorflow as tf
from tensorflow.python.client import device_lib
import pandas as pd

In [2]:
def showX(X):
    int_X = (X*255).clip(0,255).astype('uint8')
    # N*784 -> N*28*28 -> 28*N*28 -> 28 * 28N
    int_X_reshape = int_X.reshape(-1,28,28).swapaxes(0,1).reshape(28,-1)
    display(Image.fromarray(int_X_reshape))

## Data Exploration

In [5]:
with lzma.open("mnist.pkl.xz", 'rb') as f:
    train_set, validation_set, test_set = pickle.load(f, encoding='latin1')

In [6]:
train_X, train_y = train_set
validation_X, validation_y = validation_set
test_X, test_y = test_set


In [1]:
# change y to one-hot representation
train_Y = np.eye(10)[train_y]
test_Y = np.eye(10)[test_y]
validation_Y = np.eye(10)[validation_y]

NameError: name 'np' is not defined

In [5]:
# 訓練資料， X 的前 20 筆
showX(train_X[:20])
print(train_y[:20])

[5 0 4 1 9 2 1 3 1 4 3 5 3 6 1 7 2 8 6 9]


## Tensorflow

In [6]:
# 重設 session 和 graph
tf.reset_default_graph()
X = tf.placeholder(tf.float32, shape=[None, 784], name="X")
Y_ = tf.placeholder(tf.float32, shape=[None, 10], name="Y_")

In [7]:
# 設定 weight 和 bais
def weight_variable(shape):
    initial = tf.truncated_normal(shape=shape, stddev=0.1)
    return tf.Variable(initial, name ='W')
def bias_variable(shape):
    initial = tf.constant(value=0.1, shape=shape)
    return tf.Variable(initial, name = 'b')
# 設定 cnn 的 layers
def conv2d(X, W):
    return tf.nn.conv2d(X, W, strides=[1,1,1,1], padding='SAME',use_cudnn_on_gpu=True)
def max_pool_2x2(X):
    return tf.nn.max_pool(X, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

### first layer

In [8]:
# fisrt layer
with tf.name_scope('conv1'):
    ## variables
    W_conv1 = weight_variable([3,3,1,32]) # filter_height, filter_width, in_channels, out_channels
    b_conv1 = bias_variable([32])
    ## build the layer
    X_image = tf.reshape(X, [-1, 28, 28,1]) # batch, in_height, in_width, in_channels
    h_conv1 = tf.nn.relu(conv2d(X_image, W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)


### second layer

In [9]:
# second layer
with tf.name_scope('conv2'):
    ## variables
    W_conv2 = weight_variable([3,3,32,64])
    b_conv2 = bias_variable([64])
    ## build the layer
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2)

### fully-connected layer

In [10]:
# fully-connected layer
with tf.name_scope('full'):
    W_fc1 = weight_variable([7*7*64, 1024])
    b_fc1 = bias_variable([1024])
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1)+b_fc1)

### output layer

In [11]:
with tf.name_scope('dropout'):
    keep_prob = tf.placeholder("float", name="keep_prob")
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# Readout
with tf.name_scope('readout'):
    W_fc2 = weight_variable([1024,10])
    b_fc2 = bias_variable([10])
    Y = tf.matmul(h_fc1_drop, W_fc2)+b_fc2

### lose function & Optimizer

In [12]:
cross_entropy =  tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y_, logits=Y))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
prediction = tf.argmax(Y, 1, name="prediction")
correct_prediction = tf.equal(prediction, tf.argmax(Y_, 1), name="correction")
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name="accuracy")

### execution

In [13]:
device_lib.list_local_devices()

[name: "/cpu:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 274551716968468814, name: "/gpu:0"
 device_type: "GPU"
 memory_limit: 1623654400
 locality {
   bus_id: 1
 }
 incarnation: 1695376851357702308
 physical_device_desc: "device: 0, name: GeForce GT 740, pci bus id: 0000:01:00.0"]

In [14]:
sess = tf.InteractiveSession(config=tf.ConfigProto(log_device_placement=True))
tf.global_variables_initializer().run()

In [15]:
%%timeit -r 1 -n 1
with tf.device('/cpu:0'):
    for i in range(5000):
        rnd_idx = np.random.choice(train_X.shape[0], 50, replace=False)
        if i%250 == 0:
            validation_accuracy = accuracy.eval({
                    X: validation_X[:200], Y_: validation_Y[:200], keep_prob: 1.0 })
            print("step %d, validation accuracy %g"%(i, validation_accuracy))
        train_step.run({X: train_X[rnd_idx], Y_: train_Y[rnd_idx], keep_prob: 0.5 })

step 0, validation accuracy 0.07
step 250, validation accuracy 0.915
step 500, validation accuracy 0.95
step 750, validation accuracy 0.96
step 1000, validation accuracy 0.955
step 1250, validation accuracy 0.985
step 1500, validation accuracy 0.99
step 1750, validation accuracy 0.98
step 2000, validation accuracy 0.985
step 2250, validation accuracy 0.995
step 2500, validation accuracy 0.98
step 2750, validation accuracy 0.99
step 3000, validation accuracy 0.99
step 3250, validation accuracy 0.995
step 3500, validation accuracy 0.995
step 3750, validation accuracy 0.985
step 4000, validation accuracy 0.995
step 4250, validation accuracy 0.995
step 4500, validation accuracy 0.995
step 4750, validation accuracy 1
1 loop, best of 1: 2min 29s per loop


### Accuracy

In [16]:
np.mean([accuracy.eval({X: test_X[i:i+1000], 
                        Y_: test_Y[i:i+1000],
                                   keep_prob: 1.0}) 
        for i in range(0, test_X.shape[0], 1000)]
)

0.9842

### Prediction test

In [17]:
predict_raw = Y.eval(feed_dict={X: test_X[0:20],keep_prob: 1.0})
predict_raw_pd = pd.DataFrame(predict_raw)
prediction = predict_raw_pd.apply(np.argmax,axis=1)
print('predict: ',prediction.values)
showX(test_X[:20])
print('answer: ',test_y[:20])

predict:  [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]


answer:  [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]
